## Import

In [ ]:
import torch                     # for all things PyTorch
import torch.nn as nn            # for torch.nn.Module, the parent object for PyTorch models
import torch.nn.functional as F  # for the activation function

![nn](https://pytorch.org/tutorials/_images/mnist.png)

- 위의 LeNet-5의 다이어그램은 Convolutional neural nets의 가장 초기 모델 중에 하나이며, 딥러닝의 폭발적인 증가의 운전사중 하나이다.

- 이 모델은 손글씨 숫자들(the MNIST dataset)의 작은 이미지들을 읽고, 현재 이미지들을 잘 분류 할 수 있게 구축되었다.

- 여기는 어떻게 작업하는지에 대한 요약된 정보이다.:
  - Layer C1은 Convolutional 층이며, 훈련하는 동안 입력 이미지의 features들을 배우고, 이것은 Scans한다라는 의미입니다. Layer C1은 이미지에서 본 학습된 각각의 Feature들을 Map을 결과물을 산출합니다. 이 맵을 "activation Map"이고, Layer S2에서 다운 샘플링 합니다.

  - Layer C3은 다른 Convolutional 층이며, features들을 결합하기 위해서 C1의 activation map을 배로 스캐닝합니다. 또한, 이 feature의 결합들은 공간적인 위치를 설명할 수 있는 activation map으로 내보내지고, 이것들은 layer S4에서 다운 샘플링 됩니다.

  - 마지막으로, 끝에 있는 F5, F6 및 OUTPUT을 fully-connected layer 하며, 마지막 activation map을 10개의 숫자 중 하나의 빈도로 분류합니다.


- 다음으로, 심플한 neural network를 코드로 표현하는 방법 입니다.


In [ ]:
## LeNet 만들기
class LeNet(nn.Module):

  def __init__(self):
    super(LeNet, self).__init__()

    self.conv1 = nn.Conv2d(1, 6, 3)
    self.conv2 = nn.Conv2d(6, 16, 3)

    self.fc1 = nn.Linear(16 * 6 * 6, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]
    num_features = 1
    
    for s in size:
      num_features *= s

    return num_features

- 이 코드를 살펴보면, 위에 다이어그램 이미지와 이 코드의 구조가 유사하다는 것을 발견할 수 있습니다.

- 전형적인 파이토치 모델의 구조적인 모습입니다. :
  - torch.nn.Model로 부터 상속 받습니다. 모듈들은 중복 될 수 있습니다. 실제로 torch.nn.Module로 부터 Linear Layer과 Conv2d 클레스들을 상속 받습니다.

  - 모델은 init 함수를 가질 수 있으며, 레이어들을 인스턴스화 할 수 있고, 어떤 데이터를 필요에 따라서 불러올 수 있습니다.(예, NLP 모델의 학습에 필요한 어휘를 로드할 수 있다.)
  - 이 모델은 forward() 함수를 가질 수 있습니다. 여기에서 실제 계산이 발생합니다. input은 networks 층과 다양한 함수들을 통해 지나가면서 결과물을 생성합니다.
  - 그외에는, 당신은 당신의 모델을 계산하는데 필요한 서포트 방법들과 다양한 속성들의 추가, 어떤 다른 파이썬 클레스와 비슷하게 당신의 모델 클레스를 증축할 수 있습니다. 

- 이 개체를 인스턴스화하고 이를 통해 샘플 입력을 실행해 보겠습니다

In [ ]:
net = LeNet() # 인스턴스화
print(net)

input = torch.rand(1, 1, 32, 32) # 인풋 데이터 생성
print("\nImage batch shape:")
print(input)

output = net(input) # 결과 출력
print("\nRaw output::")
print(output)
print(output.shape)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Image batch shape:
tensor([[[[0.3264, 0.0723, 0.4313,  ..., 0.1648, 0.3553, 0.0566],
          [0.2730, 0.3892, 0.6041,  ..., 0.1599, 0.2777, 0.7695],
          [0.1145, 0.2277, 0.3456,  ..., 0.4742, 0.1552, 0.7626],
          ...,
          [0.1848, 0.8712, 0.1490,  ..., 0.2459, 0.6469, 0.2976],
          [0.9878, 0.7853, 0.7921,  ..., 0.5330, 0.2565, 0.0784],
          [0.3223, 0.9591, 0.9031,  ..., 0.3078, 0.8013, 0.8284]]]])

Raw output::
tensor([[ 0.0626,  0.0795, -0.0777, -0.1304, -0.0529, -0.0441,  0.0038,  0.0733,
         -0.0396, -0.0550]], grad_fn=<AddmmBackward0>)
torch.Size([1, 10])
